<a href="https://colab.research.google.com/github/pirumba/computer-vision/blob/main/annotatedmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#clone YOLOv5 and 
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

**Step 2: Assemble Our Dataset**
In order to train our custom model, we need to assemble a dataset of representative images with bounding box annotations around the objects that we want to detect. And we need our dataset to be in YOLOv5 format.

In Roboflow, you can choose between two paths:

Convert an existing dataset to YOLOv5 format. Roboflow supports over 30 formats object detection formats for conversion.
Upload raw images and annotate them in Roboflow with Roboflow Annotate.

In [ ]:
from roboflow import Roboflow
rf = Roboflow(model_format="yolov5", notebook="ultralytics")

In [ ]:
# set up environment
os.environ["DATASET_DIRECTORY"] = "/content/datasets"

In [ ]:
# #after following the link above, recieve python code with these fields filled in
# #from roboflow import Roboflow
# #rf = Roboflow(api_key="YOUR API KEY HERE")
# #project = rf.workspace().project("YOUR PROJECT")
# #dataset = project.version("YOUR VERSION").download("yolov5")
# # !pip install roboflow

# # from roboflow import Roboflow
# # rf = Roboflow(api_key="Ij15bccaCSFhYVqlPkTR")
# # project = rf.workspace().project("cropdetection-no4dz")
# # dataset = project.version(1).download("yolov5")

# # # 

# !pip install roboflow

# from roboflow import Roboflow
# rf = Roboflow(api_key="Mwu3QmrQ1Xs3zTjSBiQB")
# project = rf.workspace().project("hard-hat-sample-vkfar")
# dataset = project.version(2).download("yolov5")

# !pip install roboflow

# # from roboflow import Roboflow
# # rf = Roboflow(api_key="Mwu3QmrQ1Xs3zTjSBiQB")
# # project = rf.workspace("").project("hard-hat-sample-vkfar")
# # dataset = project.version(2).download("yolov5")

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="Mwu3QmrQ1Xs3zTjSBiQB")
project = rf.workspace().project("crops-ifvld")
dataset = project.version(1).download("yolov5")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
Image(filename = '/content/yolov5/runs/train/exp/val_batch0_pred.jpg', width =1500)

**Step 3: Train Our Custom YOLOv5 model**
Here, we are able to pass a number of arguments:

img: define input image size
batch: determine batch size
epochs: define the number of training epochs. (Note: often, 3000+ are common here!)
data: Our dataset locaiton is saved in the dataset.location
weights: specify a path to weights to start transfer learning from. Here we choose the generic COCO pretrained checkpoint.
cache: cache images for faster training

In [ ]:
!python train.py --img 416 --batch 16 --epochs 10 --data {dataset.location}/data.yaml --weights yolov5s.pt --cache

Evaluate Custom YOLOv5 Detector Performance
Training losses and performance metrics are saved to Tensorboard and also to a logfile.

If you are new to these metrics, the one you want to focus on is mAP_0.5 - learn more about mean average precision here.

In [ ]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir runs

Run Inference With Trained Weights
Run inference with a pretrained checkpoint on contents of test/images folder downloaded from Roboflow.

In [ ]:
!python detect.py --weights runs/train/exp/weights/best.pt --img 190 --conf 0.1 --source {dataset.location}/test/images

In [ ]:
#display inference on ALL test images

#display inference on ALL test images

import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/runs/detect/exp/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

In [ ]:
#export your model's weights for future use
from google.colab import files
files.download('./runs/train/exp/weights/best.pt')